# Projeto de Banco de Dados com SQLAlchemy


## **Criação do Banco de Dados**
 **Crie um banco de dados SQLite** que contenha as seguintes tabelas:
   - **Departamento** (um departamento pode ter vários funcionários).
   - **Funcionário** (um funcionário pertence a um departamento e pode ter vários projetos).
   - **Projeto** (um projeto pertence a um único funcionário).


In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# Criando a base ORM
Base = declarative_base()

In [2]:
class Departamento(Base):
    __tablename__ = "departamentos"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)

    # Relacionamento com Funcionários (One-to-Many)
    funcionarios = relationship("Funcionario", back_populates="departamento")

class Funcionario(Base):
    __tablename__ = "funcionarios"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False)
    cargo = Column(String, nullable=False)
    salario = Column(Integer, nullable=False)
    departamento_id = Column(Integer, ForeignKey("departamentos.id"))

    # Relacionamento com Departamento (Many-to-One)
    departamento = relationship("Departamento", back_populates="funcionarios")

    # Relacionamento com Projetos (One-to-Many)
    projetos = relationship("Projeto", back_populates="funcionario")

class Projeto(Base):
    __tablename__ = "projetos"

    id = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String, nullable=False, unique=True)
    descricao = Column(String)
    funcionario_id = Column(Integer, ForeignKey("funcionarios.id"))

    # Relacionamento com Funcionário (Many-to-One)
    funcionario = relationship("Funcionario", back_populates="projetos")

In [3]:
engine = create_engine("sqlite:///banco.db", echo=False) # conectou ao banco de dados banco.db caso ele exista (c.c. cria um novo)
Base.metadata.create_all(engine)

In [4]:
# Criar uma sessão
Session = sessionmaker(bind=engine)
session = Session()


## **Inserção de Dados**
**Crie e adicione os seguintes departamentos ao banco de dados:**
   - "TI"
   - "RH"
   - "Financeiro"

In [5]:
# adicionar algo a alguma tabela: session.add, depois session.commit()

In [6]:
# Criando linhas na tabela Departamento
dep1 = Departamento(nome="TI")
dep2 = Departamento(nome="RH")
dep3 = Departamento(nome="Financeiro")

session.add_all([dep1, dep2, dep3])
session.commit()

**Crie e adicione os seguintes funcionários ao banco de dados, vinculando-os a seus departamentos:**
   - "Carlos", "Desenvolvedor", salário: **5000** → Departamento: **TI**
   - "Ana", "Analista de RH", salário: **4000** → Departamento: **RH**
   - "Mariana", "Contadora", salário: **4500** → Departamento: **Financeiro**

In [7]:
func1 = Funcionario(nome="Carlos", cargo="Desenvolvedor", salario=5000, departamento=dep1)
func2 = Funcionario(nome="Ana", cargo="Analista de RH", salario=4000, departamento=dep2)
func3 = Funcionario(nome="Mariana", cargo="Contadora", salario=4500, departamento=dep3)

session.add_all([func1, func2, func3])
session.commit()

 **Crie e adicione os seguintes projetos ao banco de dados, vinculando-os a seus respectivos funcionários:**
   - "Sistema Web", descrição: **"Desenvolvimento de um sistema web"** → Funcionário: **Carlos**
   - "Treinamento Interno", descrição: **"Organização de treinamentos para funcionários"** → Funcionário: **Ana**
   - "Auditoria 2024", descrição: **"Planejamento da auditoria financeira anual"** → Funcionário: **Ana**
   - "Gestão de clientes", descrição: **"Controle e manutenção de clientes na firma"** → Funcionário: **Mariana**

In [8]:
proj1 = Projeto(nome="Sistema Web", descricao="Desenvolvimento de um sistema web", funcionario=func1)
proj2 = Projeto(nome="Treinamento Interno", descricao="Organização de treinamentos para funcionários", funcionario=func2)
proj3 = Projeto(nome="Auditoria 2024", descricao="Planejamento da auditoria financeira anual", funcionario=func2)
proj4 = Projeto(nome="Gestão de clientes", descricao="Controle e manutenção de clientes na firma", funcionario=func3)

session.add_all([proj1, proj2, proj3, proj4])
session.commit()

## **Consultando Funcionários e seus Projetos**
 **Liste todos os funcionários cadastrados, exibindo também seu cargo, o departamento ao qual pertencem e os projetos associados.**
   - Para cada funcionário, exiba:
     - Nome
     - Cargo
     - Nome do departamento
     - Lista de projetos nos quais está trabalhando, com nome e descrição.

In [9]:
funcionarios = session.query(Funcionario).all()

for func in funcionarios:
    print(func.nome, ', cargo:', func.cargo, 'dpto: ', func.departamento.nome, '\n')
    for proj in func.projetos:
        print(func.nome, ', projeto:', proj.nome, ', descrição:', proj.descricao)

Carlos , cargo: Desenvolvedor dpto:  TI 

Carlos , projeto: Sistema Web , descrição: Desenvolvimento de um sistema web
Ana , cargo: Analista de RH dpto:  RH 

Ana , projeto: Treinamento Interno , descrição: Organização de treinamentos para funcionários
Ana , projeto: Auditoria 2024 , descrição: Planejamento da auditoria financeira anual
Mariana , cargo: Contadora dpto:  Financeiro 

Mariana , projeto: Gestão de clientes , descrição: Controle e manutenção de clientes na firma


## **Consultando Funcionários por Departamento**
 **Liste todos os departamentos cadastrados e, para cada um deles, exiba os funcionários associados, incluindo o cargo e o salário.**
   - Para cada departamento, exiba:
     - Nome do departamento.
     - Lista de funcionários vinculados, com nome, cargo e salário.

In [10]:
departamentos = session.query(Departamento).all()

for dep in departamentos:
    print(f"Departamento:, {dep.nome}")
    for func in dep.funcionarios:
        print(f"Funcionário: {func.nome}, cargo: {func.cargo}, salário: {func.salario}")
    print('-' * 40)

Departamento:, TI
Funcionário: Carlos, cargo: Desenvolvedor, salário: 5000
----------------------------------------
Departamento:, RH
Funcionário: Ana, cargo: Analista de RH, salário: 4000
----------------------------------------
Departamento:, Financeiro
Funcionário: Mariana, cargo: Contadora, salário: 4500
----------------------------------------


## **Atualizações e Remoções**
**Atualize o salário do funcionário Carlos para 5500.**

In [11]:
# funcionario1 = session.query(Funcionario).filter_by(nome="Carlos").first()
# funcionario1.salario = 5000
# session.commit()

session.query(Funcionario).filter_by(nome="Carlos").update({"salario": 5500})
session.commit()

**Remova a funcionária Mariana e verifique se o projeto vinculado a ela também foi removido.**

In [12]:
funcionario_remover = session.query(Funcionario).filter_by(nome="Mariana").first()
session.delete(funcionario_remover)
session.commit()

projetos_remover = session.query(Projeto).filter_by(funcionario_id=funcionario_remover.id).all()
if not projetos_remover:
    print(f"Os projetos associados à(o) funcionária(o) {funcionario_remover.nome} foram removidos")

Os projetos associados à(o) funcionária(o) Mariana foram removidos


### **Transferindo Funcionários de departamentos**
**Transfira todos os funcionários do departamento de "TI" para o departamento de "RH".** 

In [13]:
dep_origem = session.query(Departamento).filter_by(nome="TI").first()
dep_destino = session.query(Departamento).filter_by(nome="RH").first()
funcionarios = session.query(Funcionario).filter_by(departamento_id=dep_origem.id).all()

for func in funcionarios:
    func.departamento_id = dep_destino.id

session.commit()

In [14]:
departamentos = session.query(Departamento).all()

for dep in departamentos:
    print(f"Departamento:, {dep.nome}")
    for func in dep.funcionarios:
        print(f"Funcionário: {func.nome}, cargo: {func.cargo}, salário: {func.salario}")
    print('-' * 40)

Departamento:, TI
----------------------------------------
Departamento:, RH
Funcionário: Carlos, cargo: Desenvolvedor, salário: 5500
Funcionário: Ana, cargo: Analista de RH, salário: 4000
----------------------------------------
Departamento:, Financeiro
----------------------------------------


## **Exclusão e Reset do Banco**
- **Exclua todas as tabelas do banco sem remover o arquivo.**  
- **Exclua o banco de dados completamente, removendo o arquivo SQLite.**

In [15]:
Base.metadata.drop_all(bind=engine)

In [16]:
import os

session.close()

if os.path.exists("banco.db"):
    os.remove("banco.db")
    print("Banco removido com sucesso")
else:
    print("Banco não existe")

Banco removido com sucesso
